In [1]:
import polars as pl
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from ts_utils import get_RMS_data
from tqdm.notebook import tqdm

In [2]:
dfs, _ = get_RMS_data(0, "val", normalized=False)
dfs[2]

Time,In,Un,PF_n,Irms,Urms,MeanPF,P,Q,S,Label
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64
"""2024-01-21_10-…",-0.0007,-67.138451,99.78862,0.00207,166.778534,0.930463,0.321167,0.046335,0.345169,0
"""2024-01-21_10-…",0.004374,45.454262,99.995255,0.00207,166.856877,0.930336,0.321289,0.046441,0.345347,0
"""2024-01-21_10-…",0.001983,158.539368,99.995255,0.002067,166.92651,0.930335,0.321024,0.046403,0.345063,0
"""2024-01-21_10-…",-0.001925,224.88678,71.878189,0.002067,166.908343,0.930335,0.320923,0.046389,0.344955,0
"""2024-01-21_10-…",-0.001108,225.994705,99.995255,0.002067,166.82422,0.930491,0.320851,0.04627,0.34482,0
"""2024-01-21_10-…",-0.002625,194.150055,99.998886,0.002068,166.756519,0.930224,0.320781,0.046445,0.344843,0
"""2024-01-21_10-…",-0.000817,88.911545,99.699081,0.002067,166.761899,0.930116,0.320683,0.046505,0.344778,0
"""2024-01-21_10-…",0.000583,-26.770121,99.246513,0.002068,166.834907,0.929998,0.32081,0.046605,0.344957,0
"""2024-01-21_10-…",-0.000467,-131.258621,94.309502,0.00207,166.914432,0.929996,0.321255,0.046671,0.345436,0


In [26]:
scaler = StandardScaler()
all_train_data = None
for row in tqdm(range(128)):
    dfs, _ = get_RMS_data(row, "train", normalized=False)
    for df in dfs:
        if df is None:
            continue
        df = df.drop("Time")
        df = df.to_pandas()
        if all_train_data is None:
            all_train_data = df
        else:
            all_train_data = pd.concat([all_train_data, df], axis=0)
all_train_data.head()

  0%|          | 0/128 [00:00<?, ?it/s]

,In,Un,PF_n,Irms,Urms,MeanPF,P,Q,S,Label
0,-0.001866,-108.872292,93.172928,0.002138,165.755370,0.915712,0.324536,0.057227,0.354409,0
1,0.000992,3.114024,97.170601,0.002138,165.740177,0.915747,0.324460,0.057189,0.354312,0
2,-0.002216,101.725540,96.003029,0.002138,165.744004,0.915757,0.324453,0.057180,0.354300,0
3,-0.002741,200.389496,96.003029,0.002137,165.765678,0.915777,0.324412,0.057159,0.354248,0
4,-0.002158,226.259155,96.003029,0.002137,165.780859,0.915799,0.324388,0.057139,0.354213,0


In [29]:
print(all_train_data.shape)

(63306799, 10)


In [30]:
all_train_data = all_train_data[["In", "Un", "PF_n", "Irms", "Urms", "MeanPF", "P", "Q", "S"]]
scaler.fit(all_train_data)

StandardScaler()

In [32]:
from joblib import dump
dump(scaler, "/home/mrcong/Code/mylab-nilm-files/nilm-physical-features/results/scaler/standard_scaler.joblib")

['/home/mrcong/Code/mylab-nilm-files/nilm-physical-features/results/scaler/standard_scaler.joblib']